In [525]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('data/train.csv', index_col='PassengerId')
test_df = pd.read_csv('data/test.csv', index_col='PassengerId')
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [526]:
test_df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [527]:
train_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [528]:
# mean = 0.38 => классы не сбалансированы

In [529]:
train_df.describe(include='object').T

,count,unique,top,freq
Name,891,891,"Braund, Mr. Owen Harris",1
Sex,891,2,male,577
Ticket,891,681,347082,7
Cabin,204,147,B96 B98,4
Embarked,889,3,S,644


In [530]:
traind_size = train_df.shape[0]
traind_size

891

In [531]:
test_df['Survived'] = 0

In [532]:
all_data = pd.concat([train_df, test_df], axis=0)

In [533]:
missing = pd.DataFrame(all_data.isnull().sum().sort_values(ascending=False))
missing.columns = ['count']
missing = missing.loc[(missing != 0).any(axis=1)]
missing['percent'] = missing['count'] / all_data.shape[0]
missing

,count,percent
Cabin,1014,0.774637
Age,263,0.200917
Embarked,2,0.001528
Fare,1,0.000764


In [534]:
all_data.drop('Cabin', axis=1, inplace=True)

In [535]:
all_data['Age'] = all_data['Age'].fillna(all_data['Age'].median())
all_data['Embarked'] = all_data['Embarked'].fillna(all_data['Embarked'].mode()[0])
all_data['Fare'] = all_data['Fare'].fillna(all_data['Fare'].mode()[0])

In [536]:
all_data.isnull().sum().sum()

0

In [537]:
all_data = pd.get_dummies(all_data)

In [538]:
train_df = all_data[:traind_size]
test_df = all_data[traind_size:]
test_df.drop('Survived', axis=1, inplace=True)

C:\Users\Daniil\AppData\Local\Temp\ipykernel_3732\1455086756.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop('Survived', axis=1, inplace=True)


In [539]:
y = train_df['Survived']
X = train_df.drop('Survived', axis=1)

In [540]:
from xgboost import XGBClassifier

xgb_model =  XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=2,
    max_leaves=4,
    subsample=1,
    colsample_bytree=0.6,
    random_state=42, 
    n_jobs=-1
)

xgb_model.fit(X, y)
xgb_predict = xgb_model.predict(test_df)

In [541]:
submission = {
    'PassengerId': pd.read_csv('data/test.csv').PassengerId.values,
    'Survived': xgb_predict
}
solution = pd.DataFrame(submission)
solution.to_csv('submission.csv', index=False)